In [5]:
import os 
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
import numpy as np



def plot_pareto_front(function_name, all_algorithms, num_trial=5, iterations=100, initial_number=5, paths='.',picture_paths='.',filename=None):
    point_num = num_trial * (iterations + initial_number)
    
    def getdata(lines, num_trial, iterations, initial_number):
        output = []
        stringline = []
        for i in range(num_trial):
            stringline.append(i * (iterations + initial_number + 1))
        for i in range(num_trial * (iterations + initial_number + 1)):
            if i not in stringline:
                line = lines[i]
                line = line.strip('\n')               
                line = line.split()
                output.append([ float(num) for num in line])
        output_reshape = []
        for i in range(iterations+initial_number):
            for j in range(num_trial):
                output_reshape.append(output[j * (iterations+initial_number) + i])
        output_reshape = np.array(output_reshape)
        return output_reshape

    fig = plt.figure() # Create Map
    cm = plt.get_cmap("RdYlGn")    
    col = [i for i in range(point_num)]
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    
    for i in range(len(all_algorithms)):
        algo = all_algorithms[i]
        my_file = Path(paths + function_name + '_' + algo + '_front.txt')
        if my_file.exists() == False :
            continue
        else :
            #read data
            output= open(os.path.join(paths,function_name + '_' + algo + '_front.txt'), "r")    
            lines = output.readlines()  
            output.close()
            
            if len(lines) != num_trial*(1+iterations+initial_number):
                print('wrong length')
                return
            
            output = getdata(lines, num_trial, iterations, initial_number)
            if output.shape[1] != 2:
                print('No 2D plots.')
                return
            
            #add subplot 
            location = 221 + i
            subfig = fig.add_subplot(location)
            sc1 = subfig.scatter(output[:,0], output[:,1], s=3, c=col,cmap = cm)
            plt.title(algo)
            plt.colorbar(sc1) 
    
    # save picture
    if filename is None:
        filename = function_name + ' front'
    plt.savefig(picture_paths+'\{}.png'.format(filename), dpi=400)
    plt.close()
    
def plot_hypervolume(function_name, all_algorithms, num_trial=5, iterations=100, initial_number=5, paths='.',picture_paths='.',filename=None):
    
    #hypervolume improvement
    def getdata(lines, num_trial, iterations):
        output = []
        stringline = []
        start_hv = []
        for i in range(num_trial):
            stringline.append(i * (iterations + 2))
            line = lines[i*(iterations + 2) + 1]
            line = line.strip()
            start_hv.append(float(line))
        for i in range(num_trial * (iterations + 2)):
            if i not in stringline:
                line = lines[i]
                line = line.strip()
                output.append(float(line)) #hypervolume 
        hypervolume = []
        for i in range(iterations):
            hypervolume_line = []
            for j in range(num_trial):
                hypervolume_line.append(output[j * (iterations+1) + i]-start_hv[j]) #hypervolume improvement
            hypervolume.append(hypervolume_line)
        return hypervolume
    

    def get_HPV_mean_std(output):
        HPV_diff_mean_all = []
        HPV_diff_std_all = []
        for i in range(iterations):
            HPV_diff_mean = np.mean(output[i])
            HPV_diff_std = np.std(output[i])
            HPV_diff_mean_all.append(HPV_diff_mean)
            HPV_diff_std_all.append(HPV_diff_std)
        return [HPV_diff_mean_all,HPV_diff_std_all]


    for i in range(len(all_algorithms)):
        algo = all_algorithms[i]
        #line_style = line_styles[i]
        my_file = Path(paths + function_name + '_' + algo + '_hypervolume.txt')
        if my_file.exists() == False :
            continue
        else :
            #read data
            output= open(os.path.join(paths,function_name + '_' + algo + '_hypervolume.txt'), "r")    
            lines = output.readlines()  
            output.close()
            
            if len(lines) != num_trial*(1+iterations+1):
                print('wrong length')
                return
            
            # compute mean hypervolume improvement
            HPV_improvement = getdata(lines, num_trial, iterations)

            [HPV_improvement_mean,HPV_improvement_mean_std] = get_HPV_mean_std(HPV_improvement)
            
            # plot mean hypervolume improvement
            #plt.errorbar([j+1 for j in range(iterations)], HPV_improvement_mean,yerr=HPV_improvement_mean_std, fmt=line_style,lw = 1,elinewidth=1,ms=1,capsize=1,label=algo)
            plt.plot([j+1 for j in range(iterations)], HPV_improvement_mean,lw = 1,ms=1,label=algo)
            plt.fill_between([j+1 for j in range(iterations)],np.array(HPV_improvement_mean)-0.5*np.array(HPV_improvement_mean_std),np.array(HPV_improvement_mean)+0.5*np.array(HPV_improvement_mean_std),alpha=0.2)
    plt.xlim([1,iterations])
    plt.title(function_name, fontsize=15)
    plt.xlabel('iterations', fontsize=14)
    plt.ylabel('hypervolume improvement', fontsize=14)
    plt.grid()
    plt.legend(loc='lower right')
    plt.tight_layout() 

    if filename is None:
        filename = function_name + ' hypervolume'
    plt.savefig(picture_paths+'\{}.png'.format(filename), dpi=400)
    plt.close()
    
    
  
    
    
    
    
    
    